In [34]:
import pandas as pd
import numpy as np
import os
import time
import json

from transformers import BertModel, BertTokenizer
from sklearn.preprocessing import OneHotEncoder
from flower_recommendation import recommend_flower # 우리 모델 

In [35]:
df_nlp= pd.read_csv(r"C:\Users\82104\Desktop\tave\추천시스템_데이터 (1).csv")

# JSON 문자열을 배열로 변환
df_nlp['최종_벡터'] = df_nlp['최종_벡터'].apply(lambda x: np.array(json.loads(x)))
df_nlp.head(5)

,꽃,월,계절,꽃말,설명,설명2,설명_벡터,원핫인코딩,최종_벡터
0,각시붓꽃,3,봄,"부끄러움, 세련됨",부끄러움 세련됨. 보라. 각시라 하면 이제 막 시집 온 새색시를 연상케 한다. 그래...,[CLS] 부끄러움 세련됨 [SEP] [CLS] 보라 [SEP] [CLS] 각시라 ...,"[[0.04113124683499336, 0.11827859282493591, -0...",[[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1....,"[[0.04113124683499336, 0.11827859282493591, -0..."
1,감국,10,가을,그윽한 향기,그윽한 향기. 노랑. 가을 산야는 국화과 식물들 차지다. 특히 노란 꽃으로 향기까지...,[CLS] 그윽한 향기 [SEP] [CLS] 노랑 [SEP] [CLS] 가을 산야는...,"[[0.06626041233539581, 0.14091618359088898, -0...",[[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0....,"[[0.06626041233539581, 0.14091618359088898, -0..."
2,개나리,3,봄,"희망,깊은 정, 달성",희망깊은 정 달성. 노랑. 우리나라 전역에서 봄 소식을 가장 먼저 알려주는 대표적인...,[CLS] 희망깊은 정 달성 [SEP] [CLS] 노랑 [SEP] [CLS] 우리나...,"[[0.12369333207607269, 0.1469401866197586, -0....",[[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1....,"[[0.12369333207607269, 0.1469401866197586, -0...."
3,개나리,4,봄,희망,희망. 노랑. 개나리 봄을 알리는 전령사 하면 가장 먼저 떠오르는 꽃이다. 나리나리...,[CLS] 희망 [SEP] [CLS] 노랑 [SEP] [CLS] 개나리 봄을 알리는...,"[[0.07213923335075378, 0.13994745910167694, -0...",[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1....,"[[0.07213923335075378, 0.13994745910167694, -0..."
4,갯개미취,9,가을,추억,추억. 보라. 옛날 일 따위는 깨끗이 잊는 사람들이 많은 가운데서도 당신은 옛 일을...,[CLS] 추억 [SEP] [CLS] 보라 [SEP] [CLS] 옛날 일 따위는 깨...,"[[0.23332802951335907, 0.04906110465526581, -0...",[[0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0....,"[[0.23332802951335907, 0.04906110465526581, -0..."


### 추천시스템 + 멘트 생성모델

In [43]:
import openai

# 사전에 정의된 tokenizer와 model
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertModel.from_pretrained('bert-base-multilingual-cased')
# 예시 사용자 입력
user_input = " 여자친구가 5월에 생일이야."
print(user_input)
recommended_flowers = recommend_flower(user_input, df_nlp, tokenizer, model)
print(recommended_flowers)

# 가장 높은 유사도를 가지는 꽃과 꽃말 선택
top_flower = recommended_flowers.iloc[0]
flower_name = top_flower['꽃']
flower_meaning = top_flower['꽃말']
print(top_flower)


 여자친구가 5월에 생일이야.
           꽃            꽃말       유사도
669  카푸치노 장미       첫사랑의 잔상  0.908979
817     황새냉이      그대에게 바친다  0.908213
234      모카라  청초한 아름다움, 기품  0.908104
꽃       카푸치노 장미
꽃말      첫사랑의 잔상
유사도    0.908979
Name: 669, dtype: object


In [ ]:
openai.api_key =''


# 응답 기능 생성
def get_completion(prompt, model="gpt-4"): 
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0
    )
    return response.choices[0].message['content']


user_input1 = f"너는 지금부터 감성적인 시인이며, 대답은 마치 시인이 선물하는 것처럼 해야해. 꽃과 꽃말과 사용자 입력에 따라서 가장 어울리는 꽃다발 선물 멘트를 한줄로 생성할거야. 꽃은 반드시 {flower_name}이고 꽃말은 {flower_meaning} 이야. 사용자 입력은 {user_input}이야. 멘트에 꽃과 꽃말 그리고 사용자 입력이 들어가면서 사용자 입장에서의 멘트를 구체적으로 추천해줘. 반드시 사용자 입장에서의 멘트만 간결하게 출력해야해"

# API에 쿼리
response = get_completion(user_input1)
print(f"Input: {user_input1}")
print(f"Output: {response}")